# PyArrow in Pandas

1. What is Arrow? What is PyArrow?
2. How can I use Arrow/PyArrow today?
3. Using PyArrow as a backend instead of NumPy

# What is Arrow?

Arrow is an open-source project from Apache. The idea is to have an in-memory set of data structures that are cross-platform and cross-language, for data analysis work. If you are implementing a language that wants to do data analysis, or if you're implementing a system/database/tool for data analysis, then don't re-invent the wheel -- just use Apache Arrow.

If you're using Apache Arrow, then you can probably share data with other systems using Apache Arrow.

# Well.. what's wrong with NumPy?

I've long described Pandas as an automatic transmission, whereas NumPy is the manual transmission.  Pandas has long used NumPy for storage. And that's mostly great:

- Storage is in C, so it's small and fast
- Much less memory usage than Python objects
- Vectorized operations
- We know it's rock solid

But...

- It was never designed for data storage/retrieval
- It was never designed for tabular data, like we use in Pandas
- It was never really meant for serious string operations

At some point, Arrow will replace NumPy as the backend for Pandas. It's currently experimental, but the documentation says that Arrow (PyArrow, the Python bindings for Arrow) will be mandatory in order to install Pandas 3.0 whenever it comes out.

In [1]:
# if we're using int64 in Pandas/NumPy, then an integer takes up 64 bits or 8 bytes
# how much space does a Python integer take?

x = 1000
import sys

sys.getsizeof(x)  # how many bytes will this be?

28

In [2]:
x = x ** 100

In [3]:
sys.getsizeof(x)

160

In [4]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [5]:
np.random.seed(0)
df = DataFrame(np.random.randint(0, 1000, [4,5]),
               index=list('abcd'),
               columns=list('vwxyz'))
df
               

,v,w,x,y,z
a,684,559,629,192,835
b,763,707,359,9,723
c,277,754,804,599,70
d,472,600,396,314,705


In [6]:
# how much memory does this data frame use?
# I use df.info(), which both shows me info about my columns + rows, and gives me a memory summary

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   v       4 non-null      int64
 1   w       4 non-null      int64
 2   x       4 non-null      int64
 3   y       4 non-null      int64
 4   z       4 non-null      int64
dtypes: int64(5)
memory usage: 192.0+ bytes


In [7]:
4 * 5 * 8

160

In [8]:
# because the rows and columns have strings as names, Pandas does *not* store them in NumPy!
# rather, it stores them as Python strings, and then refers to those strings from Pandas to Python memory space

df.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [9]:
df.columns

Index(['v', 'w', 'x', 'y', 'z'], dtype='object')

In [10]:
# by passing memory_usage='deep', we say: Go and find the memory usage for each string
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   v       4 non-null      int64
 1   w       4 non-null      int64
 2   x       4 non-null      int64
 3   y       4 non-null      int64
 4   z       4 non-null      int64
dtypes: int64(5)
memory usage: 360.0 bytes


In [11]:
# let's add a column

df['s'] = ['hello', 'out', 'there', 'everyone']

In [12]:
df

,v,w,x,y,z,s
a,684,559,629,192,835,hello
b,763,707,359,9,723,out
c,277,754,804,599,70,there
d,472,600,396,314,705,everyone


In [13]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v       4 non-null      int64 
 1   w       4 non-null      int64 
 2   x       4 non-null      int64 
 3   y       4 non-null      int64 
 4   z       4 non-null      int64 
 5   s       4 non-null      object
dtypes: int64(5), object(1)
memory usage: 577.0 bytes


In [15]:
df.info()  # no memory_usage = deep, and it's still off


<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v       4 non-null      int64 
 1   w       4 non-null      int64 
 2   x       4 non-null      int64 
 3   y       4 non-null      int64 
 4   z       4 non-null      int64 
 5   s       4 non-null      object
dtypes: int64(5), object(1)
memory usage: 224.0+ bytes


In [16]:
# let's assign a new string to one value

df.loc['a', 's'] = 'abcdefghij' * 10_000_000

In [17]:
df.info()  # no deep -- how much memory

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v       4 non-null      int64 
 1   w       4 non-null      int64 
 2   x       4 non-null      int64 
 3   y       4 non-null      int64 
 4   z       4 non-null      int64 
 5   s       4 non-null      object
dtypes: int64(5), object(1)
memory usage: 396.0+ bytes


In [18]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v       4 non-null      int64 
 1   w       4 non-null      int64 
 2   x       4 non-null      int64 
 3   y       4 non-null      int64 
 4   z       4 non-null      int64 
 5   s       4 non-null      object
dtypes: int64(5), object(1)
memory usage: 95.4 MB


# Using PyArrow today

We often read CSV files from disk into Pandas. A big CSV file can take a long time to load. That's because Pandas uses
a relatively slow engine for reading data.

We can replace it, with almost zero effort, with PyArrow's CSV-loading facility. 

In [19]:
filename = '/Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv'

In [20]:
!ls -lh $filename

-rw-r--r-- 1 reuven staff 656M Jun  4  2021 /Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv


In [22]:
# how long does it take to load this data into Pandas?

import time

start_time = time.time()
df = pd.read_csv(filename)
end_time = time.time()

print(f'Total is {end_time - start_time}')

Total is 11.839046955108643


In [23]:
# Let's use PyArrow instead

import time

start_time = time.time()
df = pd.read_csv(filename, engine='pyarrow')
end_time = time.time()

print(f'Total is {end_time - start_time}')

Total is 1.0558300018310547


In [24]:
df.dtypes

VendorID                         int64
tpep_pickup_datetime     datetime64[s]
tpep_dropoff_datetime    datetime64[s]
passenger_count                  int64
trip_distance                  float64
RatecodeID                       int64
store_and_fwd_flag              object
PULocationID                     int64
DOLocationID                     int64
payment_type                     int64
fare_amount                    float64
extra                          float64
mta_tax                        float64
tip_amount                     float64
tolls_amount                   float64
improvement_surcharge          float64
total_amount                   float64
congestion_surcharge           float64
dtype: object

In [26]:
type(df.values)

numpy.ndarray

# CSV is slow (because it's text)

Apache Arrow has two of its own (binary) formats that we can use for storing and retrieving data. This will always be much faster than CSV.

Arrow supports two formats:

- Feather -- faster reads/writes, but no compression (so you have larger files)
- Parquet -- slower reads/writes, but heavy compression (so you have smaller files)

In [27]:
df.to_feather('taxis.feather')

In [28]:
df.to_parquet('taxis.parquet')

In [29]:
!ls -lh taxis.*

-rw-r--r-- 1 reuven staff 254M Jun 19 11:20 taxis.feather
-rw-r--r-- 1 reuven staff 126M Jun 19 11:20 taxis.parquet


In [30]:
start_time = time.time()
df = pd.read_feather('taxis.feather')
end_time = time.time()

print(f'Total is {end_time - start_time}')

Total is 0.6063899993896484


In [31]:
start_time = time.time()
df = pd.read_parquet('taxis.parquet')
end_time = time.time()

print(f'Total is {end_time - start_time}')

Total is 2.074265956878662


In [32]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ms]
tpep_dropoff_datetime    datetime64[ms]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

# How about using PyArrow as the backend for Pandas?

The future, as I described, will eventually have PyArrow replace NumPy. We can experiment with that today, by specifying this in various ways.

In [33]:
s = Series([10, 20, 30, 40, 50])
s.dtypes

dtype('int64')

In [34]:
s = Series([10, 20, 30, 40, 50], dtype='int32')
s.dtypes

dtype('int32')

In [35]:
# we can specify that a series should use PyArrow's types
s = Series([10, 20, 30, 40, 50], dtype='int32[pyarrow]')
s.dtypes

int32[pyarrow]

In [36]:
# This can have big implications, especially now that we're using NumPy 2.0

s = Series([10, 20, 30, 40, 50], dtype='int8')
s + 120

0   -126
1   -116
2   -106
3    -96
4    -86
dtype: int8

In [37]:
s = Series([10, 20, 30, 40, 50], dtype='int8')
s + 1000  # previous versions of NumPy would identify that 1,000 needs 16 bits, promote the series, and perform the calculation

OverflowError: Python integer 1000 out of bounds for int8

In [39]:
s = Series([10, 20, 30, 40, 50], dtype='int8[pyarrow]')
s + 120

0    130
1    140
2    150
3    160
4    170
dtype: int64[pyarrow]

In [40]:
s = Series([10, 20, 30, 40, 50], dtype='int8[pyarrow]')
s + 1000

0    1010
1    1020
2    1030
3    1040
4    1050
dtype: int64[pyarrow]

In [41]:
s.values

<ArrowExtensionArray>
[10, 20, 30, 40, 50]
Length: 5, dtype: int8[pyarrow]

In [51]:
df_np = pd.read_csv(filename, engine='pyarrow')
df_np.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667792 entries, 0 to 7667791
Data columns (total 18 columns):
 #   Column                 Dtype        
---  ------                 -----        
 0   VendorID               int64        
 1   tpep_pickup_datetime   datetime64[s]
 2   tpep_dropoff_datetime  datetime64[s]
 3   passenger_count        int64        
 4   trip_distance          float64      
 5   RatecodeID             int64        
 6   store_and_fwd_flag     object       
 7   PULocationID           int64        
 8   DOLocationID           int64        
 9   payment_type           int64        
 10  fare_amount            float64      
 11  extra                  float64      
 12  mta_tax                float64      
 13  tip_amount             float64      
 14  tolls_amount           float64      
 15  improvement_surcharge  float64      
 16  total_amount           float64      
 17  congestion_surcharge   float64      
dtypes: datetime64[s](2), float64(9), int64(6),

In [52]:
# we can tell read_csv that we want to use PyArrow as our dtype_backend
# this is experimental!

df_pa = pd.read_csv(filename, engine='pyarrow', dtype_backend='pyarrow')

In [53]:
df_pa.dtypes

VendorID                        int64[pyarrow]
tpep_pickup_datetime     timestamp[s][pyarrow]
tpep_dropoff_datetime    timestamp[s][pyarrow]
passenger_count                 int64[pyarrow]
trip_distance                  double[pyarrow]
RatecodeID                      int64[pyarrow]
store_and_fwd_flag             string[pyarrow]
PULocationID                    int64[pyarrow]
DOLocationID                    int64[pyarrow]
payment_type                    int64[pyarrow]
fare_amount                    double[pyarrow]
extra                          double[pyarrow]
mta_tax                        double[pyarrow]
tip_amount                     double[pyarrow]
tolls_amount                   double[pyarrow]
improvement_surcharge          double[pyarrow]
total_amount                   double[pyarrow]
congestion_surcharge           double[pyarrow]
dtype: object

In [54]:
df_pa.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667792 entries, 0 to 7667791
Data columns (total 18 columns):
 #   Column                 Dtype                
---  ------                 -----                
 0   VendorID               int64[pyarrow]       
 1   tpep_pickup_datetime   timestamp[s][pyarrow]
 2   tpep_dropoff_datetime  timestamp[s][pyarrow]
 3   passenger_count        int64[pyarrow]       
 4   trip_distance          double[pyarrow]      
 5   RatecodeID             int64[pyarrow]       
 6   store_and_fwd_flag     string[pyarrow]      
 7   PULocationID           int64[pyarrow]       
 8   DOLocationID           int64[pyarrow]       
 9   payment_type           int64[pyarrow]       
 10  fare_amount            double[pyarrow]      
 11  extra                  double[pyarrow]      
 12  mta_tax                double[pyarrow]      
 13  tip_amount             double[pyarrow]      
 14  tolls_amount           double[pyarrow]      
 15  improvement_surcharge  double[py

In [55]:
# let's compare retrieving rows from df_np vs. df_pa



In [56]:
%timeit df_np.loc[[10, 20, 1000, 10000, 500000]]

364 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [57]:
%timeit df_pa.loc[[10, 20, 1000, 10000, 500000]]

652 ms ± 9.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
# what about retrieving columns?

%timeit df_np[['VendorID', 'passenger_count', 'trip_distance', 'total_amount']]

71.1 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
%timeit df_pa[['VendorID', 'passenger_count', 'trip_distance', 'total_amount']]

464 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
# PyArrow is faster at retrieving columns
# NumPy is (still) faster at retrieving rows

In [63]:
# let's try some queries and see how they look

In [64]:
%%timeit 

(
    df_np
    .loc[lambda df_: df_['passenger_count'] > 3]
    .loc[lambda df_: df_['trip_distance'] < 10]
    [['passenger_count', 'trip_distance']]    
)

160 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%%timeit 

(
    df_pa
    .loc[lambda df_: df_['passenger_count'] > 3]
    .loc[lambda df_: df_['trip_distance'] < 10]
    [['passenger_count', 'trip_distance']]    
)

1.35 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
# What about some basic string operations?

In [67]:
%timeit df_np['store_and_fwd_flag'].str.get(0)

2.24 s ± 240 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
%timeit df_pa['store_and_fwd_flag'].str.get(0)

176 ms ± 6.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [69]:
# grouping -- which is faster?

In [70]:
%timeit df_np.groupby(['passenger_count'])['trip_distance'].mean()

145 ms ± 7.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit df_pa.groupby(['passenger_count'])['trip_distance'].mean()

239 ms ± 3.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
